# Information Retrieval

Let's download the classical data set, i.e. the CRANFIELD text set on aeronautics

In [54]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


We take queries only (we will consider queries as documents)

In [55]:
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight


We combine  multi-string queries into one

In [56]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2] #Let's output the couple of documents as an example

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ']

### Let's make queries to our documents

In [57]:
QUERIES = ['laminar boundary', 'mixing problem']


## Boolean retrieval
Let's represent each document as a "bitmask": that is a vector with a dimensionality equal to the vocabulary size, which has 1 at every position if the document contains the corresponding term; and 0 if it does not

In [58]:
!pip install Cython

In [59]:
# in different versions the answer could also differ, therefore it's important to have the same version
! pip install -q scikit-learn==0.22.2.post1

In [60]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

Let's look at the representation of the first sentence

In [61]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

It's fine.

## Task 0

Now for each query from `QUERIES` let's find the nearest document from `query_data` according to the Jaccard similarity index. There are more effictive solutions to do it, but your task is to realize the algorithm computing the Jaccard index and then apply it to our data.

In [62]:
import numpy as np 

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  """
    Similarity or Jaccard similarity index: the ratio of the intersection cardinality to the union cardinality
  """
  # your code here

  return float(sum(vector_a & vector_b) / sum(vector_a | vector_b))

#Check that the function works correctly
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

## Task 1
Now using the code below find the most similar documents for each query.

In [63]:
for q_id, query in enumerate(encoded_queries):
  # bring to the required datatype
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  # calculate the Jaccard index
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  # sort according to it
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  # output 3 most similar documents for each query
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))



Q: laminar boundary
FOUND:
    205	0.17	has anyone investigated theoretically whether surface flexibility can stabilize a laminar boundary layer . 
    209	0.14	what are the physical significance and characteristics of separated laminar and turbulent boundary layer flows . 
    69	0.11	previous solutions to the boundary layer similarity equations . 
Q: mixing problem
FOUND:
    59	0.10	is there any simple,  but practical,  method for numerical integration of the mixing problem (i.e. the blasius problem with three-point boundary conditions) . 
    172	0.09	solution of the blasius problem with three-point boundary conditions . 
    196	0.07	the problem of similarity for representative investigation of aeroelastic effects in a flow with the absence of heating effects . 


We see that some texts intersecting with the query only in insignificant terms have a high Jaccard index (that is our ranking function).

# VSM

Now we are going to do similar calculations, but using tf-idf and cosine distance. To practice we make everything "manually", but in "real life" it's better to use existing effective solutions, e.g., cosine distance from scipy library.

In [64]:
from  sklearn.feature_extraction.text import TfidfVectorizer

# Advice: we highly recommend to check what tf-idf vectorizer
# is able to do, and change its parameters

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

## Task 2
Realize the cosine distance computation

In [74]:
from numpy.linalg import norm
from numpy import dot
import numpy as np

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  """
    Cosine distance is 1 minus the ratio of the dot product 
    and the product of L2-norm (hint: there are such norms in numpy)
  """
  # your code here
  value1 = float(dot(vector_a, vector_b))
  value2 = (norm(vector_a)*norm(vector_b))
  if value2 == 0:
    return 1
  else:
    return 1 - value1/value2


#Check that the function is working correctly
assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5


Now let's find the nearset documents to the query according to the cosine distance between the document vector and the query representation

In [75]:
for q_id, query in enumerate(tfidf_encoded_queries):
  
  # bring to the required datatype
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # Cosine distance
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # sort according to it
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: laminar boundary
FOUND:
    205	0.66	has anyone investigated theoretically whether surface flexibility can stabilize a laminar boundary layer . 
    209	0.67	what are the physical significance and characteristics of separated laminar and turbulent boundary layer flows . 
    25	0.68	what is a single approximate formula for the displacement thickness of a laminar boundary layer in compressible flow on a flat plate . 
Q: mixing problem
FOUND:
    59	0.56	is there any simple,  but practical,  method for numerical integration of the mixing problem (i.e. the blasius problem with three-point boundary conditions) . 
    16	0.79	can the three-dimensional problem of a transverse potential flow about a body of revolution be reduced to a two-dimensional problem . 
    172	0.83	solution of the blasius problem with three-point boundary conditions . 
